In [2]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go

df = pd.read_csv('docs/databases/Age-standardized suicide rates.csv')
df1 = pd.read_csv('docs/databases/IV DATASET 1.csv')
df1 = df1[['Country', 'GDI 2015', 'HDI 2015']]
df = pd.merge(df, df1, on='Country')

df['Sex'] = df['Sex'].str.replace('female', 'F').str.replace('male', 'M')

female_suicides = df[df['Sex'].str.contains('fem', case=False)]
male_suicides = df[df['Sex'].str.contains('male', case=False)]

female_suicides_2015 = female_suicides[['Country', '2015', 'GDI 2015', 'HDI 2015']]
male_suicides_2015 = male_suicides[['Country', '2015', 'GDI 2015', 'HDI 2015']]

# Merge female and male datasets on Country
merged_data = pd.merge(female_suicides_2015, male_suicides_2015, on='Country', suffixes=('_Female', '_Male'))

# Select the relevant columns for HDI, GDI, and suicide rates
data = merged_data[['Country', 'HDI 2015_Female', 'HDI 2015_Male', 'GDI 2015_Female', 'GDI 2015_Male', '2015_Female', '2015_Male']]

# Calculate trendlines for HDI (female and male)
z_hdi_female = np.polyfit(data['HDI 2015_Female'], data['2015_Female'], 1)
p_hdi_female = np.poly1d(z_hdi_female)

z_hdi_male = np.polyfit(data['HDI 2015_Male'], data['2015_Male'], 1)
p_hdi_male = np.poly1d(z_hdi_male)

# Calculate trendlines for GDI (female and male)
z_gdi_female = np.polyfit(data['GDI 2015_Female'], data['2015_Female'], 1)
p_gdi_female = np.poly1d(z_gdi_female)

z_gdi_male = np.polyfit(data['GDI 2015_Male'], data['2015_Male'], 1)
p_gdi_male = np.poly1d(z_gdi_male)

# Create plotly scatter plots for GDI and HDI
fig = go.Figure()

# Scatter plot for GDI
fig.add_trace(go.Scatter(
    x=data['GDI 2015_Female'],
    y=data['2015_Female'],
    mode='markers',
    marker=dict(color='red'),
    name='Female'
))

fig.add_trace(go.Scatter(
    x=data['GDI 2015_Male'],
    y=data['2015_Male'],
    mode='markers',
    marker=dict(color='blue'),
    name='Male'
))

# Trendline for GDI
gdi_range = np.linspace(data['GDI 2015_Female'].min(), data['GDI 2015_Female'].max(), 100)
fig.add_trace(go.Scatter(
    x=gdi_range,
    y=p_gdi_female(gdi_range),
    mode='lines',
    line=dict(color='red', dash='dash'),
    name='Female Trend'
))

fig.add_trace(go.Scatter(
    x=gdi_range,
    y=p_gdi_male(gdi_range),
    mode='lines',
    line=dict(color='blue', dash='dash'),
    name='Male Trend'
))

fig.update_layout(
    xaxis=dict(title='GDI 2015'),
    yaxis=dict(title='Average Suicide Rate (2015)'),
    title='Relationship between GDI and Suicide Rate',
    showlegend=True
)

# Create plotly scatter plots for HDI
fig2 = go.Figure()

# Scatter plot for HDI
fig2.add_trace(go.Scatter(
    x=data['HDI 2015_Female'],
    y=data['2015_Female'],
    mode='markers',
    marker=dict(color='red'),
    name='Female'
))

fig2.add_trace(go.Scatter(
    x=data['HDI 2015_Male'],
    y=data['2015_Male'],
    mode='markers',
    marker=dict(color='blue'),
    name='Male'
))

# Trendline for HDI
hdi_range = np.linspace(data['HDI 2015_Female'].min(), data['HDI 2015_Female'].max(), 100)
fig2.add_trace(go.Scatter(
    x=hdi_range,
    y=p_hdi_female(hdi_range),
    mode='lines',
    line=dict(color='red', dash='dash'),
    name='Female Trend'
))

fig2.add_trace(go.Scatter(
    x=hdi_range,
    y=p_hdi_male(hdi_range),
    mode='lines',
    line=dict(color='blue', dash='dash'),
    name='Male Trend'
))

fig2.update_layout(
    xaxis=dict(title='HDI 2015'),
    yaxis=dict(title='Average Suicide Rate (2015)'),
    title='Relationship between HDI and Suicide Rate',
    showlegend=True
)

# Display the plots
fig.show()
fig2.show()


In [2]:
import pandas as pd
import plotly.graph_objects as go

# Read the dataset
males = pd.read_csv('docs/databases/male_suicides.csv')
females = pd.read_csv('docs/databases/female_suicides.csv')

males['HDI Classification'] = males['HDI 2015'].apply(lambda x: 'HD' if x >= 0.85 else 'LD')
females['HDI Classification'] = females['HDI 2015'].apply(lambda x: 'HD' if x >= 0.85 else 'LD')
males['GDI Classification'] = males['GDI 2015'].apply(lambda x: 'HD' if x >= 0.97 else 'LD')
females['GDI Classification'] = females['GDI 2015'].apply(lambda x: 'HD' if x >= 0.97 else 'LD')

# Group the data by HDI and GDI classifications for males
hh_df_m = males[(males['HDI Classification'] == 'HD') & (males['GDI Classification'] == 'HD')]
hl_df_m = males[(males['HDI Classification'] == 'HD') & (males['GDI Classification'] == 'LD')]
lh_df_m = males[(males['HDI Classification'] == 'LD') & (males['GDI Classification'] == 'HD')]
ll_df_m = males[(males['HDI Classification'] == 'LD') & (males['GDI Classification'] == 'LD')]

# Group the data by HDI and GDI classifications for females
hh_df_f = females[(females['HDI Classification'] == 'HD') & (females['GDI Classification'] == 'HD')]
hl_df_f = females[(females['HDI Classification'] == 'HD') & (females['GDI Classification'] == 'LD')]
lh_df_f = females[(females['HDI Classification'] == 'LD') & (females['GDI Classification'] == 'HD')]
ll_df_f = females[(females['HDI Classification'] == 'LD') & (females['GDI Classification'] == 'LD')]

# Extract the necessary columns for plotting
years = ['2000', '2010', '2015']
male_hh = hh_df_m[years].mean()
male_hl = hl_df_m[years].mean()
male_lh = lh_df_m[years].mean()
male_ll = ll_df_m[years].mean()
female_hh = hh_df_f[years].mean()
female_hl = hl_df_f[years].mean()
female_lh = lh_df_f[years].mean()
female_ll = ll_df_f[years].mean()

# Define color and marker styles
colors = ['dodgerblue', 'salmon', 'limegreen', 'purple', 'blue', 'red', 'green', 'orange']
markers = ['circle', 'square', 'diamond', 'triangle-up', 'circle', 'square', 'diamond', 'triangle-up']
labels = ['Male HD-HD', 'Male HD-LD', 'Male LD-HD', 'Male LD-LD', 'Female HD-HD', 'Female HD-LD', 'Female LD-HD', 'Female LD-LD']

# Create the line plot
fig = go.Figure()

# Add traces for each category
for i, data in enumerate([male_hh, male_hl, male_lh, male_ll, female_hh, female_hl, female_lh, female_ll]):
    fig.add_trace(go.Scatter(
        x=years,
        y=data,
        mode='lines+markers',
        name=labels[i],
        marker=dict(color=colors[i], symbol=markers[i], size=8),
        line=dict(width=2)
    ))

# Set plot title and labels
fig.update_layout(
    title='Suicide Development for Male and Female for HD and LD countries',
    xaxis=dict(title='Year'),
    yaxis=dict(title='Suicide Rates'),
    legend=dict(font=dict(size=7), orientation='h', yanchor='bottom', xanchor='center', x=0.5),
    showlegend=True,
    template='plotly_white'
)

# Display the plot
fig.show()


In [3]:
import pandas as pd
import plotly.graph_objects as go

# Read the data into a DataFrame
df = pd.read_csv('docs/databases/FINAL DATASET.csv')
df = df.drop('Unnamed: 0', axis=1)

# Extract relevant columns
government_types = df['government']
hdi_values = df['HDI 2015']
democracy_status = df['democracy']
political_violence = df['political_violence']

# Grouping average HDI and political violence values by government type and democracy status
government_avg_hdi = {}
government_avg_political_violence = {}
government_democracy = {}
for i in range(len(government_types)):
    government_type = government_types[i]
    hdi_value = hdi_values[i]
    is_democracy = democracy_status[i]
    violence_value = political_violence[i]
    
    if government_type not in government_avg_hdi:
        government_avg_hdi[government_type] = []
    government_avg_hdi[government_type].append(hdi_value)
    
    if government_type not in government_avg_political_violence:
        government_avg_political_violence[government_type] = []
    government_avg_political_violence[government_type].append(violence_value)

    government_democracy[government_type] = is_democracy

# Calculate average HDI and political violence values for each government type
avg_hdi_values = [np.mean(government_avg_hdi[gt]) for gt in government_avg_hdi]
avg_political_violence_values = [np.mean(government_avg_political_violence[gt]) for gt in government_avg_political_violence]

# Sort the bars based on the average political violence values in descending order
sorted_indices = np.argsort(avg_political_violence_values)[::-1]
sorted_avg_hdi_values = [avg_hdi_values[i] for i in sorted_indices]
sorted_avg_political_violence_values = [avg_political_violence_values[i] for i in sorted_indices]
sorted_government_types = [list(government_avg_hdi.keys())[i] for i in sorted_indices]

# Create the bar plot
fig = go.Figure()

# Add the average HDI bars
fig.add_trace(go.Bar(
    x=sorted_government_types,
    y=sorted_avg_hdi_values,
    name='Average HDI',
    marker_color='dodgerblue'
))

# Add the political violence and democracy status annotations
for i, gov_type in enumerate(sorted_government_types):
    is_democracy = government_democracy[gov_type]
    if is_democracy == 1.0:
        text_label = f'D ({round(sorted_avg_political_violence_values[i], 1)})'
        color = 'green'
    else:
        text_label = f'ND ({round(sorted_avg_political_violence_values[i], 1)})'
        color = 'red'
    
    fig.add_annotation(
        x=gov_type,
        y=sorted_avg_hdi_values[i],
        text=text_label,
        showarrow=False,
        font=dict(color=color)
    )

# Set plot title and labels
fig.update_layout(
    title='Average HDI by Government / Political Violence',
    xaxis=dict(title='Government'),
    yaxis=dict(title='Average HDI'),
    template='plotly_white'
)

# Display the plot
fig.show()
